In [1]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/eval_help.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/paper_settings.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/validation_help.R"))

In [2]:
library(tidyverse)
library(xgboost)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘xgboost’

The following object is masked from ‘package:dplyr’:

    slice



### 0 - Read in prepared data sets

In [3]:
validation_ready <- readRDS(paste0(TMP_DIR, "validation-hmf-models-pdp.Rds"))

### 1 - HMF CPCT cross-validation
- CPI patients

In [4]:
run_loo_cv

function (Y_base, X_base) 
{
    loo_cv <- data.frame()
    for (id in (X_base %>% pull(patient_id))) {
        print(id)
        flush.console()
        Y_all_lr <- extract_Y(Y_base, id, "bor", "all")
        Y_all_os <- extract_Y(Y_base, id, "os", "all")
        Y_tissue_lr <- extract_Y(Y_base, id, "bor", get_tissue(X_base, 
            id))
        Y_tissue_os <- extract_Y(Y_base, id, "os", get_tissue(X_base, 
            id))
        X_all_i <- extract_X(X_base, id, "all")
        X_tissue_i <- extract_X(X_base, id, get_tissue(X_base, 
            id))
        base_lr <- grid_fit(Y_all_lr, X_all_i, parameter_grid, 
            model = "lr")$best_model
        base_os <- grid_fit(Y_all_os, X_all_i, parameter_grid, 
            model = "os")$best_model
        warm_lr <- grid_fit(Y_tissue_lr, X_tissue_i, parameter_grid, 
            model = "lr", base_model = base_lr)$best_model
        warm_os <- grid_fit(Y_tissue_os, X_tissue_i, parameter_grid, 
            model = "os", base_model = base_os)$best_model
        loo_cv <- rbind(loo_cv, get_preds_X_i(X_base, id, warm_lr, 
            warm_os))
    }
    loo_cv
}

In [10]:
set.seed(62220)

In [11]:
hmf <- validation_ready %>% drop_na(os, bor)

In [6]:
hmf_loo_cv <- data.frame()
Y_base <- hmf %>% select(all_of(c("patient_id", "tissue", "bor", "os")))

for( i in c("five_latent_purity")){
    X_base <- hmf %>% select(all_of(c("patient_id", "tissue", models[[i]])))
    hmf_loo_cv <- run_loo_cv(Y_base, X_base)
    hmf_loo_cv$model <- i
}

[1] "CPCT02010263"
[1] "CPCT02010272"
[1] "CPCT02010333"
[1] "CPCT02010356"
[1] "CPCT02010371"
[1] "CPCT02010373"
[1] "CPCT02010393"
[1] "CPCT02010436"
[1] "CPCT02010439"
[1] "CPCT02010451"
[1] "CPCT02010455"
[1] "CPCT02010469"
[1] "CPCT02010493"
[1] "CPCT02010524"
[1] "CPCT02010526"
[1] "CPCT02010530"
[1] "CPCT02010532"
[1] "CPCT02010534"
[1] "CPCT02010536"
[1] "CPCT02010550"
[1] "CPCT02010603"
[1] "CPCT02010621"
[1] "CPCT02010622"
[1] "CPCT02010632"
[1] "CPCT02010655"
[1] "CPCT02010663"
[1] "CPCT02010665"
[1] "CPCT02010667"
[1] "CPCT02010683"
[1] "CPCT02010716"
[1] "CPCT02010722"
[1] "CPCT02010727"
[1] "CPCT02010754"
[1] "CPCT02010760"
[1] "CPCT02010774"
[1] "CPCT02010779"
[1] "CPCT02010780"
[1] "CPCT02010786"
[1] "CPCT02010794"
[1] "CPCT02010801"
[1] "CPCT02010807"
[1] "CPCT02010811"
[1] "CPCT02010847"
[1] "CPCT02010852"
[1] "CPCT02010866"
[1] "CPCT02010874"
[1] "CPCT02010878"
[1] "CPCT02010882"
[1] "CPCT02010904"
[1] "CPCT02010911"
[1] "CPCT02010913"
[1] "CPCT02010918"
[1] "CPCT020

[1] "CPCT02170038"
[1] "CPCT02170039"
[1] "CPCT02180022"
[1] "CPCT02180033"
[1] "CPCT02180053"
[1] "CPCT02190043"
[1] "CPCT02210040"
[1] "CPCT02210077"
[1] "CPCT02220057"
[1] "CPCT02220075"
[1] "CPCT02230060"
[1] "CPCT02270041"
[1] "CPCT02270043"
[1] "CPCT02270045"
[1] "CPCT02270046"
[1] "CPCT02270049"
[1] "CPCT02290020"
[1] "CPCT02290061"
[1] "CPCT02300034"
[1] "CPCT02330128"
[1] "CPCT02330133"
[1] "CPCT02340001"
[1] "CPCT02340040"
[1] "CPCT02340097"
[1] "CPCT02340098"
[1] "CPCT02340107"
[1] "CPCT02390030"
[1] "CPCT02410018"
[1] "CPCT02440016"
[1] "CPCT02440038"
[1] "CPCT02010503"
[1] "CPCT02020670"
[1] "CPCT02020958"
[1] "CPCT02050276"
[1] "CPCT02080266"


### 5 - Add prediction groups 

In [7]:
hmf_loo_cv$pred_os2 <- ifelse( hmf_loo_cv$pred_os > 3, 3.1, hmf_loo_cv$pred_os)
hmf_loo_cv$lr_gp <-  cut( hmf_loo_cv$pred_lr,  breaks = c(0,.1,.5,    1) , labels = c("Low", "Medium", "High"))
hmf_loo_cv$os_gp <-  cut( hmf_loo_cv$pred_os2,  breaks = c(0,.5,1.5,   20) , labels = c("Low", "Medium", "High"))

### 6 - Send it

In [8]:
saveRDS( hmf_loo_cv, paste0(TMP_DIR, "validation-loo-cv.Rds") )